In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 或你要用的 GPU 编号
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # 关闭 TF 日志
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"  # 禁用 TF 优化，避免影响

In [1]:
import os
import requests
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from datasets import load_dataset
import transformers
from peft import get_peft_model, PrefixTuningConfig, TaskType
from transformers import Trainer, TrainingArguments
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor

In [2]:
from PIL import Image
from tqdm import tqdm
from transformers import AutoTokenizer
import numpy as np
from torchvision import transforms

def build_filtered_dataset(dataset_name='derek-thomas/ScienceQA',
                           split='train',
                           keep_grades='1-6'):
    """
    构建按年级和图像存在性过滤的数据集。

    参数:
        dataset_name (str): 数据集名称，例如 'derek-thomas/ScienceQA'。
        split (str): 数据分割，例如 'train', 'test', 'validation'。
        keep_grades (str or None): 筛选的年级段："1-6"、"7-12" 或 None 表示不过滤。

    返回:
        List[Dict]: 筛选后的样本列表。
    """

    def is_grade_allowed(grade_str):
        if keep_grades is None:
            return True
        try:
            grade_num = int(grade_str.replace("grade", ""))
            if keep_grades == "1-6":
                return 1 <= grade_num <= 6
            elif keep_grades == "7-12":
                return 7 <= grade_num <= 12
        except:
            return False
        return False



    data = load_dataset(dataset_name, split=split)
    dataset = []

    for i, sample in enumerate(data):
        try:
            if sample.get('question') is None:
                continue
            
            if sample.get("image", None) is None:
                continue

            if not is_grade_allowed(sample.get("grade", "")):
                continue

            solution = sample.get("solution", "")
            lecture = sample.get("lecture", "")
            solution_lecture = f"{solution}\n\n{lecture}".strip()
            
            image = sample["image"].convert("RGB")
            

            # image = np.array(image)
            # image = torch.tensor(image).permute(2, 0, 1)  # shape: (C, H, W)
            dataset.append({
                "image": image, 
                "question": sample["question"],
                "choices": sample["choices"],
                "hint": sample["hint"],
                "answer": sample["answer"],
                "solution_lecture": solution_lecture,
                'grade':sample["grade"],
            })
            
        except Exception as e:
            print(f"跳过第 {i} 个样本，错误：{e}")
            continue
    return dataset

dataset_train = build_filtered_dataset(split='train', keep_grades='1-6')
print(f"\n✅ 筛选后的样本数量: {len(dataset_train)}")


✅ 筛选后的样本数量: 4349


In [3]:
dataset_val = build_filtered_dataset(split='test', keep_grades='1-6')
print(f"\n✅ 筛选后的样本数量: {len(dataset_val)}")


✅ 筛选后的样本数量: 1429


In [4]:
from random import choice

sample_1 = choice(dataset_train)
print(f"Question: {sample_1['question']}")
print(f"Choices: {sample_1['choices']}")
print(f"Hint: {sample_1['hint']}")
print(f"Grade: {sample_1['grade']}")
print(f"Answer: {sample_1['answer']}")
print(f"Explanation: {sample_1['solution_lecture']}")
print(f"Image type: {type(sample_1['image'])}")


Question: Which of these states is farthest south?
Choices: ['Nebraska', 'Minnesota', 'Idaho', 'New Hampshire']
Hint: 
Grade: grade2
Answer: 0
Explanation: To find the answer, look at the compass rose. Look at which way the south arrow is pointing. Nebraska is farthest south.

Maps have four cardinal directions, or main directions. Those directions are north, south, east, and west.
A compass rose is a set of arrows that point to the cardinal directions. A compass rose usually shows only the first letter of each cardinal direction.
The north arrow points to the North Pole. On most maps, north is at the top of the map.
Image type: <class 'PIL.Image.Image'>


In [5]:
from torch.utils.data import Dataset
from PIL import Image
import torch

class QwenVLPrefixDataset(Dataset):
    def __init__(self, data_list, processor, max_label_length=256, debug=False):
        self.data_list = data_list
        self.processor = processor
        self.max_label_length = max_label_length
        self.debug = debug

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        sample = self.data_list[idx]
        return self.build_training_sample(sample)

    def build_training_sample(self, sample):
        # 1. 处理答案
        if isinstance(sample["answer"], int):
            answer_index = sample["answer"]
        else:
            answer_index = sample["choices"].index(sample["answer"])
        answer_letter = chr(65 + answer_index)

        # 2. 构建问题内容
        question_text = f"Question: {sample['question']}\nChoices:\n"
        for idx, choice in enumerate(sample["choices"]):
            question_text += f"{chr(65 + idx)}. {choice}\n"
        if sample.get("hint"):
            question_text += f"\nHint: {sample['hint']}\n"
        question_text += (
            "\nHere is a image:\n"
            "Please select the correct answer. Then, explain your reasoning in detail. "
            "Make sure your explanation is at least three sentences long, "
            "refers to specific data from the image, and shows your step-by-step logic."
        )

        # 3. 构造 chat + 图像
        image = sample["image"]
        if not isinstance(image, Image.Image):
            raise ValueError("image must be a PIL.Image.Image")
        image = image.convert("RGB").resize((224, 224))

        chat = [
            {"role": "user", "content": [
                {"type": "text", "text": question_text},
                {"type": "image","image": image}
            ]}
        ]
        prompt = self.processor.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

        # 4. 编码图文输入（注意不要设置 truncation 或 max_length）
        inputs = self.processor(
            text=prompt,
            images=image,
            return_tensors="pt",
            padding="longest",   # ✅ 自动对齐
            truncation=False     # ✅ 不截断任何 token
        )

        # 5. 编码标签
        label_text = f"Answer: {answer_letter}\nExplanation: {sample['solution_lecture']}"
        tokenizer = self.processor.tokenizer
        
        input_len = inputs["input_ids"].shape[1]
        
        label_ids = tokenizer(
            label_text,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=input_len
        )["input_ids"]
        label_ids[label_ids == tokenizer.pad_token_id] = -100

        # 6. 返回项（确保 input_ids 和 attention_mask 等长）
        input_ids = inputs["input_ids"].squeeze(0)
        attention_mask = inputs["attention_mask"].squeeze(0)
        if input_ids.shape != attention_mask.shape:
            raise ValueError(f"input_ids shape {input_ids.shape} ≠ attention_mask shape {attention_mask.shape}")

        result = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": label_ids.squeeze(0),
            "pixel_values": inputs["pixel_values"].squeeze(0),
        }

        if "image_grid_thw" in inputs:
            result["image_grid_thw"] = inputs["image_grid_thw"].squeeze(0)

        if self.debug:
            print(f"[DEBUG] input_ids: {input_ids.shape}")
            print(f"[DEBUG] attention_mask: {attention_mask.shape}")

        return result


In [6]:
from transformers import AutoProcessor
from torch.utils.data import DataLoader

# 假设你已经有了 data_list，每个元素包含 image（路径或PIL对象）、question、choices、hint、answer、solution_lecture
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")
train_dataset = QwenVLPrefixDataset(dataset_train, processor, debug=False)
val_dataset = QwenVLPrefixDataset(dataset_val, processor, debug=False)
dataloader = DataLoader(dataset_train, batch_size=6, shuffle=True)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [7]:
import torch
from torch.nn.utils.rnn import pad_sequence

def qwen_vl_collate_fn(batch):
    """
    可自动 pad 的 collate 函数，适配不同长度 input_ids / labels。
    """
    def pad_tensor_list(tensor_list, pad_value=0):
        return pad_sequence(tensor_list, batch_first=True, padding_value=pad_value)

    input_ids = [item["input_ids"] for item in batch]
    attention_mask = [item["attention_mask"] for item in batch]
    labels = [item["labels"] for item in batch]
    pixel_values = [item["pixel_values"] for item in batch]  # 通常 shape 一致，可直接 stack
    image_grid_thw = [item["image_grid_thw"] for item in batch]  # 通常 shape 一致，可直接 stack

    input_ids = pad_tensor_list(input_ids, pad_value=0)
    attention_mask = pad_tensor_list(attention_mask, pad_value=0)
    labels = pad_tensor_list(labels, pad_value=-100)  # 对 labels padding 用 -100 避免影响 loss
    pixel_values = torch.stack(pixel_values)
    image_grid_thw = torch.stack(image_grid_thw)
    return {
        "input_ids": input_ids,
        # "attention_mask": attention_mask,
        "labels": labels,
        "pixel_values": pixel_values,
        'image_grid_thw': image_grid_thw
        
    }

In [8]:
from torch.utils.data import DataLoader

# 正确使用 collate_fn

dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=qwen_vl_collate_fn)

# 获取第一个 batch
first_batch = next(iter(dataloader))

# 查看第一个样本的结构
print("==== 第一个 batch 的第一个样本 ====")
print(f"input_ids shape: {first_batch['input_ids'][0].shape}")
# print(f"attention_mask shape: {first_batch['attention_mask'][0].shape}")
print(f"labels shape: {first_batch['labels'][0].shape}")
print(f"pixel_values shape: {first_batch['pixel_values'][0].shape}")
print(f"image_grid_thw shape: {first_batch['image_grid_thw'][0].shape}")

# 可选：查看文本内容（需要 tokenizer）
tokenizer = processor.tokenizer
decoded_input = tokenizer.decode(first_batch['input_ids'][0], skip_special_tokens=True)
decoded_label = tokenizer.decode(
    [id for id in first_batch['labels'][0].tolist() if id != -100],
    skip_special_tokens=True
)

print("\n--- 解码后的 input_ids ---")
print(decoded_input)

print("\n--- 解码后的 labels ---")
print(decoded_label)


==== 第一个 batch 的第一个样本 ====
input_ids shape: torch.Size([162])
labels shape: torch.Size([162])
pixel_values shape: torch.Size([256, 1176])
image_grid_thw shape: torch.Size([3])

--- 解码后的 input_ids ---
system
You are a helpful assistant.
user
Question: What is the name of the colony shown?
Choices:
A. Maryland
B. New Hampshire
C. Rhode Island
D. Vermont

Here is a image:
Please select the correct answer. Then, explain your reasoning in detail. Make sure your explanation is at least three sentences long, refers to specific data from the image, and shows your step-by-step logic.
assistant


--- 解码后的 labels ---
Answer: B
Explanation: The colony is New Hampshire.
During the colonial era, New Hampshire and New York both claimed the territory that would later become the state of Vermont. Vermont was never its own colony.


In [ ]:
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", device_map="auto", torch_dtype=torch.bfloat16
)


In [ ]:
peft_config = PrefixTuningConfig(
    task_type='CAUSAL_LM',
    inference_mode=False,
    num_virtual_tokens=12,
    prefix_projection=True
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [9]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  # 屏蔽 num_items_in_batch
        if "num_items_in_batch" in kwargs:
            kwargs.pop("num_items_in_batch")
        outputs = model(**inputs)
        loss = outputs.loss
        return (loss, outputs) if return_outputs else loss

In [10]:
import re
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import numpy as np

def parse_output(output):
    output = output.strip()

    # case 1: "Answer: A Explanation: xxx"
    match = re.search(r"Answer[:：]?\s*([A-D])\b.*?Explanation[:：]?\s*(.+)", output, re.DOTALL)
    if match:
        answer = ord(match.group(1)) - 65
        explanation = match.group(2).strip()
        return answer, explanation

    # case 2: "A Explanation: xxx"
    match = re.match(r"\b([A-D])\s*Explanation[:：]?\s*(.+)", output, re.DOTALL)
    if match:
        answer = ord(match.group(1)) - 65
        explanation = match.group(2).strip()
        return answer, explanation

    # case 3: "A. xxx" or "B: xxx"
    match = re.match(r"\b([A-D])[\.:]\s*(.+)", output, re.DOTALL)
    if match:
        answer = ord(match.group(1)) - 65
        explanation = match.group(2).strip()
        return answer, explanation

    # case 4: only one letter like "C"
    match = re.match(r"^\s*([A-D])\s*$", output)
    if match:
        answer = ord(match.group(1)) - 65
        return answer, ""

    # fallback: try to find first capital letter A-D (unsafe)
    match = re.search(r"\b([A-D])\b", output)
    if match:
        answer = ord(match.group(1)) - 65
        explanation = output[match.end():].strip()
        return answer, explanation

    return -1, ""

def keyword_overlap(pred, ref):
    pred_keywords = set(pred.lower().split())
    ref_keywords = set(ref.lower().split())
    if not ref_keywords:
        return 0.0
    return len(pred_keywords & ref_keywords) / len(ref_keywords)

MAX_LEN = 1024
smoothie = SmoothingFunction().method4

import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import torch
import gc

# ==== 全局设置 ====
MAX_LEN = 512
smoothie = SmoothingFunction().method4
global_rouge = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

# ==== Keyword Overlap ====
def keyword_overlap(pred, ref):
    pred_keywords = set(pred.lower().split())
    ref_keywords = set(ref.lower().split())
    if not ref_keywords:
        return 0.0
    return len(pred_keywords & ref_keywords) / len(ref_keywords)

# ==== Metric 函数 ====
def compute_metrics(eval_preds, tokenizer):
    predictions = eval_preds.predictions
    label_ids = eval_preds.label_ids
    inputs = eval_preds.inputs if hasattr(eval_preds, "inputs") else None  # 如果你想一起打印输入

    if isinstance(predictions, tuple):
        predictions = predictions[0]
    if predictions.ndim == 3:
        predictions = predictions.argmax(-1)

    decoded_preds = []
    decoded_labels = []

    print("\n================= Sample Predictions =================")
    for i, (pred_ids, label) in enumerate(zip(predictions, label_ids)):
        try:
            label = [id for id in label if id != -100]
            if hasattr(pred_ids, "tolist"):
                pred_ids = pred_ids.tolist()

            decoded_pred = tokenizer.decode(pred_ids, skip_special_tokens=True)
            decoded_label = tokenizer.decode(label, skip_special_tokens=True)

            decoded_pred = decoded_pred.encode("utf-8", errors="ignore").decode("utf-8", errors="ignore")[:MAX_LEN].strip()
            decoded_label = decoded_label.encode("utf-8", errors="ignore").decode("utf-8", errors="ignore")[:MAX_LEN].strip()

            if not decoded_pred or not decoded_label:
                continue

            decoded_preds.append(decoded_pred)
            decoded_labels.append(decoded_label)

            # 👉 打印输入输出对（可选加输入）
            print(f"[{i}]")
            if inputs is not None and "input_ids" in inputs:
                input_ids = inputs["input_ids"][i].tolist()
                decoded_input = tokenizer.decode(input_ids, skip_special_tokens=True)
                print(f"🟡 Input: {decoded_input}")
            print(f"🔵 Label: {decoded_label}")
            print(f"🟢 Pred : {decoded_pred}\n")

        except Exception as e:
            print(f"[❌ Decode error at sample {i}] {e}")
            continue

    # === Metric ===
    bleu1_scores = []
    bleu4_scores = []
    rouge_l_scores = []
    keyword_overlaps = []
    choice_correct = []

    for i, (pred, label) in enumerate(zip(decoded_preds, decoded_labels)):
        try:
            reference = label.split()
            candidate = pred.split()

            if not reference or not candidate:
                continue

            bleu1 = sentence_bleu([reference], candidate, weights=(1, 0, 0, 0), smoothing_function=smoothie)
            bleu4 = sentence_bleu([reference], candidate, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
            rouge_l = global_rouge.score(label, pred)["rougeL"].fmeasure
            keyword_acc = keyword_overlap(pred, label)

            pred_choice, _ = parse_output(pred)
            label_choice, _ = parse_output(label)
            correct = int(pred_choice == label_choice and pred_choice != -1)

            bleu1_scores.append(bleu1)
            bleu4_scores.append(bleu4)
            rouge_l_scores.append(rouge_l)
            keyword_overlaps.append(keyword_acc)
            choice_correct.append(correct)

        except Exception as e:
            print(f"[❌ Metric error at sample {i}] {e}")
            continue

    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return {
        "BLEU-1": np.mean(bleu1_scores) if bleu1_scores else 0.0,
        "BLEU-4": np.mean(bleu4_scores) if bleu4_scores else 0.0,
        "ROUGE-L": np.mean(rouge_l_scores) if rouge_l_scores else 0.0,
        "KeywordOverlap": np.mean(keyword_overlaps) if keyword_overlaps else 0.0,
        "ChoiceAccuracy": np.mean(choice_correct) if choice_correct else 0.0,
    }



In [15]:
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")
small_train_dataset=dataset_train
small_val_dataset=dataset_val
train_dataset = QwenVLPrefixDataset(small_train_dataset, processor, debug=False)
val_dataset = QwenVLPrefixDataset(small_val_dataset, processor, debug=False)    
# dataloader = DataLoader(train_dataset, batch_size=6, shuffle=True)

In [19]:
from transformers import Trainer, TrainingArguments
import torch



training_args = TrainingArguments(
    output_dir="./qwen2.5vl-prefix2.03B",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1, 
    dataloader_num_workers=0,
    eval_accumulation_steps=1,
    learning_rate=5e-4,
    num_train_epochs=5,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    bf16=True,  # 如果使用的是支持 bfloat16 的 GPU，可改为 bf16=True
    gradient_accumulation_steps=4,
    do_eval=True,
    remove_unused_columns=False
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=qwen_vl_collate_fn,
    compute_metrics=lambda p: compute_metrics(p, tokenizer=processor.tokenizer)
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
[codecarbon ERROR @ 18:14:06] Error: Another instance of codecarbon is probably running as we find `/tmp/.codecarbon.lock`. Turn off the other instance to be able to run this one or use `allow_multiple_runs` or delete the file. Exiting.


In [ ]:
# trainer.train()

In [12]:
from transformers import AutoModelForVision2Seq
from peft import PeftModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 第一步：加载 base 模型（视觉语言模型）
base_model =  AutoModelForVision2Seq.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct",
    device_map={"": device},
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

# 第二步：加载 Prefix Tuning 权重
prefix_path = "./qwen2.5vl-prefix2.03B/checkpoint-1360"  # 比如 checkpoint-1360
model = PeftModel.from_pretrained(base_model, prefix_path)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
trainer.model = model

In [ ]:
# metrics = trainer.evaluate()

In [ ]:
# print(metrics)

In [17]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
from IPython.display import clear_output
import numpy as np

def evaluate_step_by_step(model, dataset, tokenizer, collate_fn, batch_size=1, device="cuda"):
    model.eval()
    dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)

    bleu1_scores = []
    bleu4_scores = []
    rouge_l_scores = []
    keyword_overlaps = []
    choice_correct = []

    for i, batch in enumerate(tqdm(dataloader)):
        # Move to device
        batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
        
        with torch.no_grad():
            outputs = model(**batch)

        # 获取 logits → token ids
        if hasattr(outputs, "logits"):
            pred_ids = outputs.logits.argmax(-1).detach().cpu().tolist()
        else:
            continue

        label_ids = batch["labels"].detach().cpu().tolist()

        # decode
        for j, (pred, label) in enumerate(zip(pred_ids, label_ids)):
            pred_text = tokenizer.decode(pred, skip_special_tokens=True)
            label_text = tokenizer.decode([id for id in label if id != -100], skip_special_tokens=True)

            ref = label_text.split()
            cand = pred_text.split()
            if not ref or not cand:
                continue

            # 计算各项指标
            bleu1 = sentence_bleu([ref], cand, weights=(1, 0, 0, 0), smoothing_function=smoothie)
            bleu4 = sentence_bleu([ref], cand, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
            rouge_l = global_rouge.score(label_text, pred_text)["rougeL"].fmeasure
            keyword_acc = keyword_overlap(pred_text, label_text)

            pred_choice, _ = parse_output(pred_text)
            label_choice, _ = parse_output(label_text)
            correct = int(pred_choice == label_choice and pred_choice != -1)

            # 记录
            bleu1_scores.append(bleu1)
            bleu4_scores.append(bleu4)
            rouge_l_scores.append(rouge_l)
            keyword_overlaps.append(keyword_acc)
            choice_correct.append(correct)

            # 每条输出一条，清空上一条
            clear_output(wait=True)
            print(f"\n[Sample {i * batch_size + j}]")
            print(f"🔹BLEU-1: {bleu1:.4f}  BLEU-4: {bleu4:.4f}")
            print(f"🔹ROUGE-L: {rouge_l:.4f}  KeywordOverlap: {keyword_acc:.4f}")
            print(f"🔹ChoiceAccuracy: {correct}")
            print(f"🔸Reference: {label_text}")
            print(f"🔸Prediction: {pred_text}")

    # 返回平均分
    return {
        "BLEU-1": np.mean(bleu1_scores) if bleu1_scores else 0.0,
        "BLEU-4": np.mean(bleu4_scores) if bleu4_scores else 0.0,
        "ROUGE-L": np.mean(rouge_l_scores) if rouge_l_scores else 0.0,
        "KeywordOverlap": np.mean(keyword_overlaps) if keyword_overlaps else 0.0,
        "ChoiceAccuracy": np.mean(choice_correct) if choice_correct else 0.0,
    }



In [18]:
evaluate_step_by_step(model, val_dataset, processor.tokenizer, qwen_vl_collate_fn, batch_size=1)


100%|██████████| 1429/1429 [02:35<00:00,  9.17it/s]


[Sample 1428]
🔹BLEU-1: 0.1371  BLEU-4: 0.0169
🔹ROUGE-L: 0.1857  KeywordOverlap: 0.3000
🔹ChoiceAccuracy: 0
🔸Reference: Answer: A
Explanation: The bulldozer pushes the loose dirt. The direction of the push is away from the bulldozer.

A force is a push or a pull that one object applies to another. Every force has a direction.
The direction of a push is away from the object that is pushing.
The direction of a pull is toward the object that is pulling.
🔸Prediction: : B
Explanation: Look is the capital of the... rose. it.. is. the. a force push force direction direction a direction direction direction direction direction. a a direction push direction direction of a a a push direction. direction direction a a of a of a a direction a a a a a a a a a a object. a of a object. a of the direction a a a direction a a is a a a a a a a the a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a of a a a a a a a a a a a a a a a a a a a a of a a a a a a a a a a a a a a a a a a a a a a a force f

{'BLEU-1': 0.22413028480130578,
 'BLEU-4': 0.07225036086702381,
 'ROUGE-L': 0.2542470650297796,
 'KeywordOverlap': 0.3902395612075573,
 'ChoiceAccuracy': 0.3547935619314206}